# Import and Load

In [1]:
import sqlite3
import pandas as pd


In [2]:
conn = sqlite3.connect('../database.db')

In [3]:
df_report = pd.read_sql_query('''
    SELECT
    rl.community,
    rl."type",
    rl.beds,
    rl.sq_feet, 
    rl.price,
    rl.latitude,
    rl.longitude,
    PERCENT_RANK() OVER (ORDER BY c.crime_count) AS crime_percentile,
    (SELECT s.name FROM schools s
        JOIN school_ranking sr ON sr.school_id = s.school_id
        WHERE sr.school_id = swwz.school_id
        ORDER BY sr.school_rating DESC
        LIMIT 1) AS highest_rated_school_name,
    MAX(sr.school_rating) AS highest_school_rating,    
    CASE 
        WHEN sl.school_id IS NOT NULL THEN 'Required'
        ELSE 'Not Required'
    END AS lottery_requirement,
    ('https://www.rentfaster.ca'|| rl.link) AS link
FROM
    rental_listings rl
    INNER JOIN schools_within_walk_zone swwz ON swwz.listing_id = rl.id
    INNER JOIN listing_with_crime lwc ON lwc.listing_id = rl.id
    INNER JOIN community_crime c ON c.id = lwc.crime_id
    INNER JOIN schools s ON s.school_id = swwz.school_id
    INNER JOIN school_ranking sr ON sr.school_id = s.school_id
    LEFT JOIN school_lottery sl ON sl.school_id = s.school_id 
WHERE
    sr.school_group = 'elementary'
    AND rl.price <= 2100
    AND rl.is_active = True
GROUP BY
    rl.id
HAVING highest_school_rating >= 8
ORDER BY
    highest_school_rating DESC,
    price ASC
                              ''',conn)

#
#

In [4]:
conn.close()

In [5]:
df_report.columns

Index(['community', 'type', 'beds', 'sq_feet', 'price', 'latitude',
       'longitude', 'crime_percentile', 'highest_rated_school_name',
       'highest_school_rating', 'lottery_requirement', 'link'],
      dtype='object')

In [6]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
df_report[['community', 'type', 'beds', 'sq_feet', 'price', 'crime_percentile',
       'highest_rated_school_name', 'highest_school_rating',
       'lottery_requirement', 'link']]['community'].unique()

array(['Sunalta', 'Beltline', 'Scarboro', 'Oakridge', 'Cedarbrae',
       'Montgomery', 'Bowness', 'Braeside', 'Bayview', 'Sunnyside',
       'Hillhurst', 'Aspen Woods', 'Glenbrook', 'Glamorgan',
       'Rutland Park', 'Currie Barracks', 'Lincoln Park', 'Briar Hill',
       'St Andrews Heights', 'Westgate', 'Elboya', 'Rosscarrock',
       'Parkhill-Stanley Park', 'Windsor Park', 'Wildwood',
       'Meadowlark Park', 'Manchester', 'Spruce Cliff', 'Stanley Park'],
      dtype=object)

# Plot listings on map

In [7]:
import folium
from folium.plugins import Geocoder

# Create a map centered at the average location
m = folium.Map(location=[df_report['latitude'].mean(), df_report['longitude'].mean()], zoom_start=10)

Geocoder().add_to(m)

# Add points to the map
for idx, row in df_report.iterrows():
    # Create the popup text
    popup_text = folium.Popup(f"""
        <b>Community:</b> {row['community']}<br>
        <b>Type:</b> {row['type']}<br>
        <b>Beds:</b> {row['beds']}<br>
        <b>Sq Feet:</b> {row['sq_feet']}<br>
        <b>Price:</b> {row['price']}<br>
        <b>Crime Percentile:</b> {row['crime_percentile']}<br>
        <b>School Name:</b> {row['highest_rated_school_name']}<br>
        <b>School Rating:</b> {row['highest_school_rating']}<br>
        <b>Lottery Requirement:</b> {row['lottery_requirement']}<br>
        <b>URL:</b> <a href="{row['link']}" target="_blank">View Listing</a>
    """, max_width=450)

    # Add a marker to the map with the popup
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=popup_text
    ).add_to(m)

# Save the map to an HTML file
m.save('map.html')

In [8]:
m